# Problem 5 Jupyter Helper

# Building a Trie in Python

Before we start let us reiterate the key components of a Trie or Prefix Tree. A trie is a tree-like data structure that stores a dynamic set of strings. Tries are commonly used to facilitate operations like predictive text or autocomplete features on mobile phones or web search.

Before we move into the autocomplete function we need to create a working trie for storing strings.  We will create two classes:
* A `Trie` class that contains the root node (empty string)
* A `TrieNode` class that exposes the general functionality of the Trie, like inserting a word or finding the node which represents a prefix.

Give it a try by implementing the `TrieNode` and `Trie` classes below!

In [1]:
## Represents a single node in the Trie
class TrieNode:
    def __init__(self):
        ## Initialize this node in the Trie
        self.is_word = False
        self.children = {}
    
    def insert_char(self, char):
        ## Add a child node in this Trie
        assert type(char) == str, f'''Inserted character must be string!'''
        assert len(char) <= 1, f'''
        The length of the character must be less than or equal to 1!
        '''
        self.children[char] = TrieNode()

In [2]:
## The Trie itself containing the root node and insert/find functions
class Trie:
    def __init__(self):
        ## Initialize this Trie (add a root node)
        self.root = TrieNode()

    def insert_word(self, word):
        ## Add a word to the Trie
        current_node = self.root
        
        word = word.lower()
        for char in word:
            if char not in current_node.children:
                current_node.insert_char(char)
            current_node = current_node.children[char]
        
        current_node.is_word = True

    def find(self, prefix):
        ## Find the Trie node that represents this prefix
        current_node = self.root
        
        prefix = prefix.lower()
        for char in prefix:
            if char not in current_node.children:
                return None
            current_node = current_node.children[char]
        
        return current_node

# Finding Suffixes

Now that we have a functioning Trie, we need to add the ability to list suffixes to implement our autocomplete feature.  To do that, we need to implement a new function on the `TrieNode` object that will return all complete word suffixes that exist below it in the trie.  For example, if our Trie contains the words `["fun", "function", "factory"]` and we ask for suffixes from the `f` node, we would expect to receive `["un", "unction", "actory"]` back from `node.suffixes()`.

Using the code you wrote for the `TrieNode` above, try to add the suffixes function below. (Hint: recurse down the trie, collecting suffixes as you go.)

In [3]:
class TrieNode:
    def __init__(self):
        ## Initialize this node in the Trie
        self.is_word = False
        self.children = {}
    
    def insert_char(self, char):
        ## Add a child node in this Trie
        assert type(char) == str, f'''Inserted character must be string!'''
        assert len(char) <= 1, f'''
        The length of the character must be less than or equal to 1!
        '''
        char = char.lower()
        self.children[char] = TrieNode()
    
    def has_children(self):
        return len(self.children) > 0
    
    def blank_set(self):
        empty_set = set()
        return empty_set
    
    def recurse_suffix_helper(self, node = None, suffix = "",
                              suffix_set = None, debug_mode = False):
        '''
        Helper Function Purpose: The purpose of this helper function is to 
            find all the suffixes below a given prefix that create a word.
            The function recursively calls itself in order to search 
            through all the nodes.
            
        Arguments: 
            Node = This is the node that's having it's suffix examined.
                It is initialized as "None", but is updated as the function
                is recursively called.
            Suffix = This is the character string being examined. It is 
                initialized as none, but is updated when the function is
                called recursively.
            Suffix_set = This is the set of suffixes that make a word for
                a particular prefix. This is initialized as None but is 
                updated when the funciton is called recursively.
            Debug_mode = This prints debugging text when it is set to true,
                but by default it is set to false.
        
        Returns: Output_collection = A set of all the suffixes in the Trie 
            that make a complete word for a particular prefix.
        '''
        # Debugging is initialized for the purpose of seeing bugs in
        # the function.
        debugging = debug_mode
        
        # This is a base case in which nothing follows a prefix.
        if node == None:
            if not self.has_children():
                return self.blank_set()
        
        # If we are not looking at the very first node, and the
        # current suffix is a word, then we add it to the suffix_set.
        if (node != None) and (node.is_word == True):
            
            # If debug mode is true, this prints the suffix if 
            # it can be formed into a word.
            if debug_mode:
                print(f'''
                The suffix ({suffix}) forms a word. Adding to set of
                suffixes!
                ''')
            
            # The suffix is added to the suffix set.
            suffix_set.add(suffix)
            
            # If debug mode is on, this prints the current suffix set.
            if debug_mode:
                print(f'''
                The set of suffixes is now: 
                ({suffix_set})
                ''')
        
        # If we are not looking at the initial node and the current node
        # does not have children. The suffix_set is returned.
        if (node != None) and (not node.has_children()):
            return suffix_set
    
        # This initiallizes the variables for the for loop that
        # calls the recurse_suffix_helper function to iterate through
        # all the suffixes to find the suffixes that make complete words
        # for a given prefix.
        output_collection = self.blank_set()
        if node == None:
            char_dict = self.children
        else:
            char_dict = node.children
            
        # This for loop iterates through the suffixes by calling upon
        # recurse_suffix_helper recursively.
        for char in char_dict:
            # This creates suffix_tracker to be put into
            # recurse_suffix_helper for the suffix_set argument.
            if node == None:
                suffix_tracker = self.blank_set()
            else:
                suffix_tracker = suffix_set
                
            # This creates new_suffix to be put into
            # recurse_suffix_helper for the suffix argument.
            new_suffix = suffix + char
            
            # This creates the node to be put into
            # recurse_suffix_helper for the node argument.
            if node is None:
                new_node = self.children[char]
            else:
                new_node = node.children[char]
            
            # The output is set to the recursive function.
            output = self.recurse_suffix_helper(new_node, new_suffix,
                                                suffix_tracker, debugging)
            
            # Once the output is actualized, the function is done
            # recursing, if there is anything in the output, it is appended
            # to the output_collection set.
            if len(output) >= 1:
                for i in output:
                    output_collection.add(i)
                    
        # Once the for loop is over, the output collection set is returned.
        return output_collection
        
    def suffixes(self):
        ## Recursive function that collects the suffix for 
        ## all complete words below this point
        output = self.recurse_suffix_helper()
        output = list(output)
        output.sort()
        return output

# Testing it all out

Run the following code to add some words to your trie and then use the interactive search box to see what your code returns.

In [4]:
MyTrie = Trie()
wordList = [
    "ant", "anthology", "antagonist", "antonym", 
    "fun", "function", "factory", 
    "trie", "trigger", "trigonometry", "tripod"
]
for word in wordList:
    MyTrie.insert_word(word)

In [5]:
a_word = MyTrie.find("a")

In [6]:
a_word.recurse_suffix_helper()

{'nt', 'ntagonist', 'nthology', 'ntonym'}

In [7]:
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import interact
def f(prefix):
    if prefix != '':
        prefixNode = MyTrie.find(prefix)
        if prefixNode:
            print('\n'.join(prefixNode.suffixes()))
        else:
            print(prefix + " not found")
    else:
        print('')
interact(f,prefix='');

interactive(children=(Text(value='', description='prefix'), Output()), _dom_classes=('widget-interact',))

In [8]:
# REFERENCES
# 1. Udacity Data Structures & Algorithms Nanodegree; 3) Basic Algorithms; 
#        1) Basic Algorithms; 6) Tries
# 2. Udacity Data Structures & Algorithms Nanodegree; 3) Basic Algorithms; 
#        4) Problems Vs Algorithms; 6) Problem 5: Autocomplete with Tries